In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D

In [12]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train =  x_train [...,tf.newaxis]
x_test =  x_test [...,tf.newaxis]

In [13]:
type(x_train[0][0,0,0])

numpy.float64

In [14]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [15]:
class Model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32,3,activation="relu")
    self.flatten = Flatten()
    self.dense1 = Dense(128,activation="relu")
    self.dense2 = Dense(10)

  def call(self,x):
     x = self.conv1(x)
     x = self.flatten(x)
     x = self.dense1(x)
     x = self.dense2(x)
     return x

model = Model()

In [16]:
loss_functio = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [17]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [18]:
@tf.function
def train(imagse,labels):
    with tf.GradientTape() as tape:
        pred = model(imagse)
        loss = loss_functio(labels,pred)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,pred)

In [19]:
@tf.function
def test(imagse,labels):
        pred = model(imagse)
        loss = loss_functio(labels,pred)
        test_loss(loss)
        test_accuracy(labels,pred)

In [22]:
epochs = 6
for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for imagse , labels in train_data:
    train(imagse,labels)

  for imagse , labels in test_data:
    test(imagse,labels)

  print("epoch :" ,epoch + 1,
        f"Train Loss :{train_loss.result()}",
        f"Train accuracy:{train_accuracy.result()}",
        f"Test Loss :   {test_loss.result()}",
        f"Test accuracy:{test_accuracy.result()}")


epoch : 1 Train Loss :0.013233217410743237 Train accuracy:0.9957500100135803 Test Loss :   0.05517441779375076 Test accuracy:0.9839000105857849
epoch : 2 Train Loss :0.010493618436157703 Train accuracy:0.9963333606719971 Test Loss :   0.060939472168684006 Test accuracy:0.9836000204086304
epoch : 3 Train Loss :0.006334445904940367 Train accuracy:0.9978333115577698 Test Loss :   0.07295738905668259 Test accuracy:0.9824000000953674
epoch : 4 Train Loss :0.006924728862941265 Train accuracy:0.9978166818618774 Test Loss :   0.07743749767541885 Test accuracy:0.9818000197410583
epoch : 5 Train Loss :0.0055735171772539616 Train accuracy:0.9982333183288574 Test Loss :   0.07204453647136688 Test accuracy:0.9847999811172485
epoch : 6 Train Loss :0.00373273529112339 Train accuracy:0.9988833069801331 Test Loss :   0.08158309757709503 Test accuracy:0.9839000105857849


In [25]:
import cv2
import numpy as np
image = cv2.imread("sample_image.png")
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image = cv2.resize(image,(28,28))
image = image[...,tf.newaxis]
image = image [tf.newaxis,...]
image = image.astype("float32")

prad = model(image)
result = np.argmax(prad)
result

7

In [26]:
model.save("model")
model = tf.keras.models.load_model("model")